# Planning the Algorithm

Need to be able to sort the input matrices to prepare, for cuBLAS, arrays for each of the ms, ks, ns, and lds etc. Also, we need an array of groupsizes; essentially an array of integers, where the integers correspond to the group count of each sub-group.

The typical way random matrices composed of blocks can be created is by generating a random 'edges' array and using this array to define the starts and stops of blocks in corresponding diffuse and source matrices.

In [2]:
import cupy as cp

In [16]:
#first define necessary problem parameters
n_ant = 6
n_eig = 2   #change these as necesssary
n_src = 1
n_gains = 2*n_ant   #Re Im split
ant_1_array, ant_2_array = cp.tril_indices(n_ant, k=-1)   #indices of antennas used for calibration
# print(ant_1_array)

n_bl = 2*len(ant_1_array)    #number of baselines (no autos ie. bl_i - bl_i correlations)

In [17]:
edges = cp.unique(cp.random.randint(1, int(n_bl / 2) - 1, size = (n_ant))*2)
print(edges)

[ 2  6 16 24 26]


In [18]:
print(edges)

[ 2  6 16 24 26]


In [19]:
noise = cp.random.rand(n_bl)
diffuse = cp.random.rand(n_bl, n_eig)
source = cp.random.rand(n_bl, n_src)

In [20]:
print('\nSparse noise:\n \n', noise)

dense_noise = cp.diag(noise)
print('\nDense noise: \n \n', dense_noise)


Sparse noise:
 
 [0.95747048 0.19473865 0.15646363 0.00903927 0.83676643 0.99908792
 0.69240962 0.83536957 0.98937624 0.67943621 0.32095761 0.60088618
 0.89967188 0.93543926 0.10590774 0.88664167 0.66938198 0.56193471
 0.44964428 0.25254415 0.66485324 0.27643903 0.56904713 0.31843665
 0.09807633 0.18128298 0.38437274 0.27477038 0.20937541 0.93881057]

Dense noise: 
 
 [[0.95747048 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.19473865 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.      

In [ ]:
import numpy as np
tmp = noise[..., None]*diffuse
# tmp2 = cp.transpose(diffuse, [0, 2, 1])@tmp


print(tmp[edges[1]:edges[2]])


[[0.61653733 0.1199625 ]
 [0.13106551 0.29761171]
 [0.56454442 0.34351153]
 [0.30831974 0.41895938]
 [0.00224475 0.18500195]
 [0.41341889 0.45908363]
 [0.17883994 0.66301996]
 [0.11271723 0.38220909]
 [0.09319688 0.03032246]
 [0.60502194 0.23730035]]


In [32]:
a = np.array([1,2,3])
b = np.array([[1,2], 
              [1,2],
              [1,2]])

print(a,'\n', b)

c = a[..., None]*b

[1 2 3] 
 [[1 2]
 [1 2]
 [1 2]]


In [33]:
print(c)

[[1 2]
 [2 4]
 [3 6]]


## Matrix multiply type and group sizes

We need to be able to perform grouped batched multiplies because of terms like 

$$\Delta^{\dagger} N^{-1}  \Delta$$

that condense blocks with shapes $N_{group \ size} \times N_{eig}$ into shapes with $N_{eig} \times N_{eig}$. In other words, this is the routine that 'makes' many small blocks. The general function, however, is a heterogeneous block matrix multiply.

In order to perform this using cuBLAS gemmGroupedBatched, we must prepare the following:

1. Arrays of size groupCount (Ms, Ns, Ks, lda, etc....). In our case, the number of groups can be quickly recovered using the length of the edges array.

2. Arrays of pointers to the matrices within each group. These are the matrices defined by redundant blocks of equal size.

3. Organize the blocks such that blocks of the same size are 'together'.

### Python-side set up

On the python side of things, our job is to prepare the data so that it can be easily fed to the ctypes wrapper around the grouped batched cublas function. The main priority is to use the edges array to sort and group together blocks of equal size, then to assign those blocks to their own arrays. We will need to keep track of the groupings of the groups of blocks to be able to quickly re-assemble the matrix product post-mutliplication. Though, this can likely be done straightforwardly by sorting the edges array or defining a new $\textit{sorted\_edges}$ array that groups together blocks of equal sizes.